**This code is running in sequence**. Please run the code one block by one block so you can easily grasp the concept. For the detail, you can read the readme.md  
Try to **take a quick look at Loan-Data.csv**, so we know what data we will process and how to clean it.

## Importing the Packages

In [2]:
import numpy as np

Code di bawah ini hanya untuk ***how we see the value on screen*** dan tidak memengaruhi hasil perhitungan.

In [3]:
np.set_printoptions(suppress = True, linewidth = 100, precision =2)

# supress = stop NumPy to using scientific notation to express numbers
# linewidth = extend the number of chars we fit in a single line output to 100
# precision = display only two digits after the decimal point

## Importing the Data

In [4]:
raw_data = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_header = 1,   # First row contains table heading 
                         autostrip = True)  # Remove excess white space which can distort our columns
raw_data

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

Kita bisa mengetahui **terdapat 8805 data yang NaN** pada raw_data dengan isnan

In [5]:
np.isnan(raw_data).sum()

88005

Seperti pada *4.11 Pre-processing Data with Numpy* kita akan **membuat 2 variabel sebagai filler data NaN**  
Yaitu **temporary_fill** (nilai maksimum dari suatu dataset)  
dan **temporary_mean** (rata-rata sementara sebelum NaN dicleansing)

In [6]:
temporary_fill = np.nanmax(raw_data) + 1
temporary_mean = np.nanmean(raw_data, axis = 0)

C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\671849336.py:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data, axis = 0)


Akan muncul **RuntimeWarning** yang memberitahu bahwa *ada sesuatu di nan.mean*, yaitu "mean of empty slice"  
NumPy memberitahu "Ada kolom yang meannya NaN nih, bener ga datanya?"  
**Yang berarti ada suatu kolom yang isinya memang string semua**. Karena tidak mungkin dari banyak data pada kolom tersebut, semuanya data kosong

In [7]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

Bisa dibuktikan, ada beberapa kolom yang meannya memang NaN. **Karenanya, kita akan splitting data**.  
Akan ada array yang khusus menampung kolom string dan ada yang khusus menampung angka  
Tapi sebelum itu, **kita harus membuat temporary_stats** yang berisi nilai minimum, mean, dan maksimum sementara sebelum datanya diisi dengan filler yang kita buat di *temporary_fill*

In [8]:
temporary_stats = np.array([np.nanmin(raw_data, axis = 0),
                            temporary_mean,
                            np.nanmax(raw_data, axis = 0)])

C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\1976739372.py:1: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data, axis = 0),
C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\1976739372.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data, axis = 0)])


Abaikan RuntimeWarning, karena kita run data yang memang terdapat string  
**Output temporary_stats adalah array dua dimensi**, stack pertama berisi nanmin setiap kolom, diikuti nanmean, dan nanmax

In [9]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## Splitting the Dataset

### Splitting the Columns

Kita harus tahu lebih dulu, mana kolom yang berisi string dan mana yang numeric. Dengan cara **mencari index kolom tersebut dengan *argwhere***

In [10]:
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

Kolom yang berisi string adalah kolom kedua, keempat dan seterusnya *(meannya NaN)*  
Sedangkan yang berisi numerik adalah kolom pertama, ketiga dan seterusnya

In [11]:
columns_numerics = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numerics

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

### Re-importing the Dataset

Kemudian, kita re-import datasetnya untuk split berdasarkan string dan numerical. Disini kita **menggunakan usecols berdasarkan indexing yang sudah dibuat sebelumnya**

In [12]:
loan_data_strings = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_header = 1, autostrip = True,
                         usecols = columns_strings,  # Data yang digunakan hanya pada kolom yang berisi string saja
                         dtype = str)  # Ubah dtypenya menjadi string untuk memastikan
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [13]:
loan_data_numerics = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_header = 1, autostrip = True,
                         usecols = columns_numerics, # Data yang digunakan hanya pada kolom yang berisi numeric saja
                         filling_values = temporary_fill)  # Untuk filler NaN, menggunakan temp_fill yang sudah dibuat diatas
loan_data_numerics

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

### The Names of the Columns

Karena sebelumnya skip_header, sekarang **kita harus menambahkan kembali headernya**. Karena tidak mungkin file yang mau dianalis atau diproses tidak memiliki header.  

In [14]:
header_full = np.genfromtxt("Loan-Data.csv", delimiter = ";", 
                         skip_footer = raw_data.shape[0],  # Kita hanya menggunakan baris pertama saja, sedangkan isi datanya tidak 
                         autostrip = True,
                         dtype = str)  
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

Untuk memudahkan, **masing-masing tipe data dibuat variabel header sendiri** berdasarkan indexing sebelumnya

In [15]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numerics]

In [16]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [17]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Creating Checkpoints:

Ini tidak ada di tutorial NumPy, namun Checkpoint adalah teknik penting saat pre-processing   
**Fungsinya untuk save progress sementara**, sebagai penjagaan jika kita tidak sengaja override nama variabel dan kesalahan lainnya  


In [18]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

Kita membuat **variabel dengan nama "checkpoint"** yang memiliki tiga parameter, file_name, header, dan data  
Lalu menggunakan bantuan **npz dan npy** untuk storingnya  
Kemudian variabel **checkpoint_variable yang berfungsi untuk load npz filenya**

In [19]:
checkpoint_test = checkpoint("test", header_strings, loan_data_strings)

Kita tes dengan memasukkan header dan data dari kolom string

In [20]:
checkpoint_test["header"]

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [21]:
checkpoint_test["data"]

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

# **Manipulating** String Columns

Langkah utama dalam pre-processing adalah membersihkan setiap kolom yang ada di data  
Penjelasan masing-masing atribut kolom ada di file ***Loan Dataset Dictionary.xlsx***  
Langkah pertama adalah kita akan membersihkan kolom yang berisi String terlebih dahulu

## Issue Date

Dimulai dari kolom pertama **issue_date**

In [22]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

Dari array tersebut nama kolom *issue_d* masih terdapat typo  
Mari kita ubah menjadi **issue_date**

In [23]:
header_strings[0] = "issue_date"

Lalu kita analisis isi dari kolom *issue_date* dengan **np.unique**

In [24]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

Bisa dilihat, pola dari datanya sama semua. Yaitu nama bulan yang ada di tahun 2015.  
Karenanya **kita bisa extract nama bulannya saja** tanpa tahunnya dengan **np.chararray.strip**

In [25]:
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")

In [26]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

Biasanya dalam analisis, bulan akan didefinisikan sebagai *integer* agar tidak memakan memori  
Buat variabel **months** yang berisi unique value dari kolom *issue_date* dengan urutan sebenarnya

In [27]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

Assign value *issue_date* sebelumnya menjadi urutan *numerical value* dari array *months* dengan **for loop**  
Ubah Januari menjadi 1, Februari menjadi 2, dst. Sedangkan **NaN menjadi 0**

In [28]:
for i in range(13):
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])

In [29]:
np.unique(loan_data_strings[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

Dataset di *issue_date* sudah dalam numerical value namun masih dalam format string  
**Kita akan convert semuanya ke string di akhir pre-processing**

In [30]:
checkpoint_1 = checkpoint("issue_date", header_strings, loan_data_strings)

## Loan Status

Kolom **Loan_Status** mempunyai ketentuan dari tim Data bahwa mereka hanya mau melihat **dua kategori** saja  
Yaitu "Apakah klien mempunyai kondisi finansial yang **baik** atau **buruk** ?"

In [31]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [32]:
np.unique(loan_data_strings[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

Sedangkan terdapat 9 kategori di Loan_Status. Kita sebagai analis diberi ketentuan tambahan bahwa  
**Kat.Baik**  = Current, Fully Paid, In Grace Period, Issued, dan Late (16-30 days)  
**Kat.Buruk** = Charged Off, Default, Late (31-120 days), dan NaN   

Maka kita kodifikasi **Baik = 1** sedangkan **Buruk = 0**

In [33]:
status_bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

Lalu assign valuenya dengan bantuan np.**isin** dan np.**where**  
Jika isi array Loan_Status sama dengan isi array "status_bad" maka assign dengan 0  
Jika tidak, maka assign 1

In [34]:
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad),0,1)

In [35]:
np.unique(loan_data_strings[:,1])

array(['0', '1'], dtype='<U69')

In [36]:
checkpoint_1 = checkpoint("loan_status", header_strings, loan_data_strings)

## Term

Sekarang kita akan pre-process Kolom **Term**

In [37]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [38]:
np.unique(loan_data_strings[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

Kita bisa substract **months** dari Term sehingga tersisa jumlah bulannya saja

In [39]:
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], " months")
loan_data_strings[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

Lalu buat nama kolom menjadi lebih spesifik yang memberitahu value di dalamnya adalah jumlah bulan

In [40]:
header_strings[2] = "term_months"

Ketentuan untuk setiap data NaN adalah **"perkirakan yang terburuk"**  
Maka data NaN akan direplace dengan value 60 bulan

In [41]:
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '', 
                                  '60', 
                                  loan_data_strings[:,2])
loan_data_strings[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [42]:
np.unique(loan_data_strings[:,2])

array(['36', '60'], dtype='<U69')

## Grade and Subgrade

Kedua kolom ini terlihat mirip, mari kita analisis lebih lanjut

In [43]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [44]:
np.unique(loan_data_strings[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [45]:
np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

Setelah dilihat, isi dari **Subgrade adalah turunan dari Grade**. Lalu, kenapa kita harus punya dua kolom?  
Karena *redundant* maka **kita jadikan jadi satu kolom saja**. Tapi tidak boleh sembarangan merge ya!

### Filling Sub Grade

SubGrade ada yang NaN, untuk mengisinya kita harus "**perkirakan yang terburuk**" namun tetap dengan analisa  
Jika SubGradenya NaN, tetapi data tersebut punya Grade; maka Subgradenya diisi dengan value terburuk dari Grade tersebut

***Contoh :***  
Subgradenya NaN, namun Gradenya B  
Maka Subgradenya diisi dengan "B5"

In [46]:
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

# [1:] pada code tersebut berarti "proses data yang mempunyai grade saja" karena index 0 pada unique value kolom ketiga adalah NaN

In [47]:
np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

Ternyata kita masih punya missing data yang tidak punya subgrade ataupun grade  
Mari kita **lihat perbandingan count dari missing data tersebut dengan data keseluruhan**

In [48]:
np.unique(loan_data_strings[:,4], return_counts = True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5],
       dtype=int64))

Ternyata missing valuenya hanya 9. *Statistically Speaking* jika dibandingkan dengan keseluruhan data tidak berpengaruh signifikan  
Maka kita akan **hapus record semua data yang missing** dengan cara buat kategori baru untuk data tersebut yaitu **H1**

In [49]:
loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                  'H1',
                                  loan_data_strings[:,4])

In [50]:
np.unique(loan_data_strings[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

### Removing Grade

Karena Subgrade sudah menampung semua data yang dibutuhkan, maka Grade sudah tidak diperlukan  
**Hapus isi kolom Grade beserta headernya**

In [51]:
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

In [52]:
loan_data_strings[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [53]:
header_strings = np.delete(header_strings, 3)

In [54]:
header_strings[3]

'sub_grade'

### Converting Sub Grade

Seperti kolom sebelumnya, kita **kodifikasi setiap kategori di SubGrade menjadi angka**  
Karena valuenya banyak, kita tidak akan menggunakan np.where  
Namun akan menggunakan for loop dengan bantuan **dictionary** dan **zip**

In [55]:
np.unique(loan_data_strings[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

Buat list yang berisi **keys** dan **values**  
Valuesnya berisi fungsi **range** dengan start 1, dan stop berdasarkan posisi urutannya di unique value

Untuk lebih jelasnya fungsi dict dan zip **bisa ditonton disini**  
https://www.youtube.com/watch?v=qj-V2Ep4coY

In [56]:
keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1))  
# + 1 diakhir agar semua values termasuk. Ingat indexing di Python open closed (value terakhir tidak dimasukkan)
dict_sub_grade = dict(zip(keys, values))

In [57]:
dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

Setelah dibuat dictionary, maka for loop sudah bisa bekerja

In [58]:
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])

In [59]:
np.unique(loan_data_strings[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

## Verification Status

Langkah yang sama untuk **Verification Status**

In [60]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [61]:
np.unique(loan_data_strings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

Setelah melihat dokumentasi dari tim Data, mereka juga ingin Verifikasi terdiri dari 2 kode saja  
**0** untuk NaN dan Not Verified  
**1** untuk Source Verified dan Verified

In [62]:
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') | (loan_data_strings[:,4] == 'Not Verified'), 0 ,1)

In [63]:
np.unique(loan_data_strings[:,4])

array(['0', '1'], dtype='<U69')

## URL

Lalu kita analisis isi dari kolom **URL**

In [64]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [65]:
np.unique(loan_data_strings[:,5])

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=12606806',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=13026045',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1312426', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8138291',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8214572',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=849994'], dtype='<U69')

URL nya terlihat pengulangan, dan hanya dibedakan oleh id dibelakangnya  
Jika kita lihat keseluruhan data, di **numeric terdapat kolom id**  
Mari kita lihat apakah kolom URL menyimpan data unique atau tidak penting untuk dianalisis

In [66]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [67]:
loan_data_strings[:,5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

Bandingkan dengan data yang ada di **kolom numeric id**

In [68]:
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [69]:
loan_data_numerics[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

Terlihat sekilas sama, tapi tidak boleh menggunakan "perasaan" dalam menganalisa data  
Bandingkan dengan operator **array_equal**

Pertama, kita harus **convert dulu URL tersebut ke integer**

In [70]:
loan_data_strings[:,5].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

Pastikan id juga memiliki format int32 agar sama dengan URL

In [71]:
loan_data_numerics[:,0].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

Lalu bandingkan keduanya

In [72]:
np.array_equal(loan_data_numerics[:,0].astype(dtype = np.int32), loan_data_strings[:,5].astype(dtype = np.int32))

True

Hasilnya **True** berarti kedua data identik sama  
Maka kita akan **menghapus kolom URL karena tidak menyimpan insightful data** untuk di analisa lebih lanjut tim Data

In [73]:
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings, 5)

## State Address

Terakhir adalah kolom **State Address** dan masih terdapat typo di penamaan kolomnya

In [74]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [75]:
header_strings[5] = "state_address"

In [76]:
np.unique(loan_data_strings[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

Terdapat **50 inisial state** pada dataset kita yang mana sesuai dengan inisial semua state di Amerika Serikat  
**Kecuali Iowa** yang dijadikan sebagai *benchmark modelling* oleh tim Data Science

In [77]:
np.unique(loan_data_strings[:,5], return_counts = True)

(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27], dtype=int64))

Untuk melihat data agar lebih insigthful, kita harus mengurutkannya berdasarkan **count**  
Sedangkan NumPy secara otomatis mengurutkannya berdasarkan *alphabetical* dari inisial state

Maka kita buat urutan berdasarkan count dengan :  
1. Buat variabel ***states_names*** dan ***states_counts*** untuk menampung inisial dan count  
2. Buat variabel ***states_counts_sorted*** untuk menampung urutan count dari yang paling besar ke kecil
3. *states_names* menggunakan indexing dari *states_counts_sorted*

In [78]:
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10], dtype=int64))

Ternyata **data yang NaN** atau tidak memiliki inisial state **lebih banyak dari 45 state lainnya**  
Jika dilanjutkan dengan modelling tim Data Science, maka **outliers akan sangat memengaruhi analisa** 45 state tersebut

Setelah berdiskusi dengan tim DS, maka **seluruh states akan digrouping berdasarkan 4 kategori** Badan Sensus Amerika  
Agar hasil dari modelling dipengaruhi oleh outliers dengan tingkatan minimum  
**Referensi** Badan Sensus Amerika : https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [79]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

Sedangkan untuk **NaN akan dibuat kodifikasi 0**

In [80]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '', 
                                  0, 
                                  loan_data_strings[:,5])

**Data lainnya juga diminta kodifikasi** dengan urutan West 1, South 2, Midwest 3, dan East 4 

In [81]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [82]:
np.unique(loan_data_strings[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

# Converting to Numbers

Setelah semua proses String selesai, maka selanjutnya adalah **mengubah tipe data string tersebut menjadi integer** 

In [83]:
loan_data_strings

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [84]:
loan_data_strings = loan_data_strings = loan_data_strings.astype(np.int32)

In [85]:
loan_data_strings

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

## **Checkpoint** Strings

Setelah selesai suatu proses, practicalnya adalah **membuat checkpoint sebagai safeline**

In [86]:
checkpoint_strings = checkpoint("Checkpoint-Strings", header_strings, loan_data_strings)

In [87]:
checkpoint_strings["header"]

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [88]:
checkpoint_strings["data"]

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

Untuk memastikan apakah data yang ada di checkpoint sama dengan data yang kita proses gunanan **array_equal**

In [89]:
np.array_equal(checkpoint_strings['data'], loan_data_strings)

True

# Manipulating **Numeric Columns**

Berikutnya adalah pre-processing semua numeric columns yang ada di **loan_data_numerics**

In [90]:
loan_data_numerics

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

Kemudian kita cek **apakah ada data yang NaN**?

In [91]:
np.isnan(loan_data_numerics).sum()

0

Tidak ada data yang NaN. Kenapa?   
Karena pada proses **Splitting the Dataset** > **Re-importing the Dataset** kita sudah mengisi NaN dengan *temporary fill*  

Dan berdasarkan arahan dari tim Data Science, untuk semua data numeric NaN harus diisi dengan ***worst case scenario***  
Karenanya kita akan replace kembali temporary fill tersebut dengan data yang seharusnya

## **Substitute** "Filler" Values

In [92]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

Untuk mencari data yang sebelumnya NaN, kita harus **mencari data yang valuenya = *temporary fill***

### ID

Seharusnya kolom ID tidak ada yang NaN, karena di database pengguna harus memiliki ID untuk memiliki akses

In [93]:
temporary_fill

68616520.0

In [94]:
np.isin(loan_data_numerics[:,0], temporary_fill).sum()

0

Dan dapat dibuktikan, **tidak ada temporary fill di kolom ID**

### **Temporary Stats**

Setelah bertanya ke tim bisnis, apa worst case scenario dari masing-masing kolom?  
Mereka menjawab **semua kolom selain funded_amount dikategorikan buruk jika jumlahnya sangat kecil**  
**Hanya funded_amount**, dikategorikan buruk jika jumlahnya sangat **besar**

Karenanya seorang Data Analis harus selalu berkomunikasi dengan tim bisnis yang terkait dan tim Data yang akan mengolah lebih lanjut data yang sudah dibersihkan

In [95]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

Sebelumnya kita sudah membuat **temporary_stats dari raw data** yang belum diberi filler  
Sekarang kita akan menggunakan stats tersebut **untuk replace temp fill**  
dan hanya berfokus pada penggunaan stats columns_numerics

In [96]:
temporary_stats[:, columns_numerics]

# columns_numerics adalah variabel yang berisi indexing untuk numeric columns
# lihat di Splitting Dataset > Splitting the Columns

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

Urutan barisnya adalah min, mean, lalu max

### Funded Amount

In [97]:
loan_data_numerics[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

Sesuai dengan arahan tim bisnis, NaN values di kolom ini direplace dengan nilai minimumnya

In [98]:
loan_data_numerics[:,2] = np.where(loan_data_numerics[:,2] == temporary_fill, 
                                  temporary_stats[0, columns_numerics[2]],
                                  loan_data_numerics[:,2])
loan_data_numerics[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

### Loaned Amount, Interest Rate, Total Payment, and Installment

Untuk empat kolom ini, kita akan **replace NaN value (temp_fill) dengan nilai maksimumnya**

In [99]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [100]:
for i in [1,3,4,5]:
    loan_data_numerics[:,i] = np.where(loan_data_numerics[:,i] == temporary_fill,
                                      temporary_stats[2, columns_numerics[i]],
                                      loan_data_numerics[:,i])

In [101]:
loan_data_numerics

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

## Currency **Change**

Seperti yang dijelaskan di readme.md kita akan convert USD ke EUR dan juga sebaliknya  
Karena perusahaan kita ada di Uni Eropa, namun untuk transaksi tetap mengacu pada US Dollar

### The Exchange Rate

Tim bisnis memberikan **EUR-USD.csv** sebagai acuan exchange rate  
Terdiri dari 5 kolom, namun yang digunakan hanya kolom **Close** yaitu exchange rate saat transaksi terakhir dilakukan **di bulan** tersebut. Data di file csv tersebut sudah diurutkan berdasarkan bulannya. Jadi kita tinggal memasukkan saja

Tujuan dari langkah ini adalah **membuat kolom baru yang bernama Exchange_Rate yang akan digabungkan dengan data utama**

In [102]:
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

Kita akan menggunakan kolom **issue_date** dari load_data_strings untuk menentukan exhange rate pada suatu bulan  

In [104]:
exchange_rate = loan_data_strings[:,0]
exchange_rate

array([ 5,  0,  9, ...,  6,  4, 12])

Ketentuan dari tim bisnis adalah **jika issue_datenya NaN maka menggunakan mean dari Exchange Rate semua bulan**

In [105]:
for i in range(1,13):  
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],  
                             exchange_rate) 

# Kode diatas untuk issue_date yang tidak NaN  
# 0 adalah data NaN, dan 13 tidak ikut dihitung (jumlah bulan ada 12)
# -1 karena indexing di python dimulai dari 0, jika tidak digunakan maka Januari akan menggunakan Exchange Rate Februari

for i in range(1,13):  
    exchange_rate = np.where(exchange_rate == 0,
                             np.mean(EUR_USD),  
                             exchange_rate) 
    
# Kode diatas untuk issue_date yang NaN  
exchange_rate 

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

Untuk menggabungkannya, kita harus **menyamakan shape dari kedua data**

In [106]:
exchange_rate.shape

(10000,)

In [108]:
loan_data_numerics.shape

(10000, 6)

Bentuknya berbeda. Maka kita **np.reshape** exchange rate sesuai dengan shape dari data utama  
Menjadi **dua dimensional**

In [109]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

Lalu digabungkan dengan **np.hstack** agar menjadi kolom baru di paling kanan data utama

In [111]:
loan_data_numerics = np.hstack((loan_data_numerics, exchange_rate))

Yang terakhir adalah memberikan **header kolom pada data exchange rate** dengan **np.concatenate**

In [112]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

# Kenapa exhange_rate harus diletakkan di dalam np.array?
# Karena header_numeric adalah array satu dimensional
# Jika tidak di dalam np.array, maka akan error karena jumlah dimensinya berbeda 

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

### From USD to EUR

In [ ]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

#### Expanding the header

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

### Interest Rate

### Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## Creating the "Complete" Dataset

## Sorting the New Dataset

## Storing the New Dataset